In [2]:
import numpy as np
import pandas as pd
import torch

In [3]:
url = "https://raw.githubusercontent.com/PRABHAV-RA2011003011010/Graduate_Admission_Prediction/refs/heads/main/Admission_Predict.csv"
df = pd.read_csv(url)
print(df.head())

   Serial No.  GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  \
0           1        337          118                  4  4.5   4.5  9.65   
1           2        324          107                  4  4.0   4.5  8.87   
2           3        316          104                  3  3.0   3.5  8.00   
3           4        322          110                  3  3.5   2.5  8.67   
4           5        314          103                  2  2.0   3.0  8.21   

   Research  Chance of Admit   
0         1              0.92  
1         1              0.76  
2         1              0.72  
3         1              0.80  
4         0              0.65  


In [4]:
df.shape

(400, 9)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         400 non-null    int64  
 1   GRE Score          400 non-null    int64  
 2   TOEFL Score        400 non-null    int64  
 3   University Rating  400 non-null    int64  
 4   SOP                400 non-null    float64
 5   LOR                400 non-null    float64
 6   CGPA               400 non-null    float64
 7   Research           400 non-null    int64  
 8   Chance of Admit    400 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 28.3 KB


In [6]:
df.duplicated().sum()

np.int64(0)

In [7]:
df.drop(columns=['Serial No.'],inplace=True)

In [8]:
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


In [9]:
X=df.iloc[:,0:-1]
Y=df.iloc[:,-1]

In [10]:
X

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
395,324,110,3,3.5,3.5,9.04,1
396,325,107,3,3.0,3.5,9.11,1
397,330,116,4,5.0,4.5,9.45,1
398,312,103,3,3.5,4.0,8.78,0


In [11]:
Y

,Chance of Admit
0,0.92
1,0.76
2,0.72
3,0.80
4,0.65
...,...
395,0.82
396,0.84
397,0.91
398,0.67


In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=1)

In [13]:
X_train.shape

(320, 7)

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

X_train_scaled= scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [15]:
X_train_scaled[0]

array([0.22      , 0.17857143, 0.25      , 0.5       , 0.42857143,
       0.25      , 1.        ])

Covert Numpy arrays to Pytorch tensors

In [16]:
X_train_tensor = torch.from_numpy(X_train.values.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.values.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.values.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.values.astype(np.float32))

In [17]:
y_train_tensor

tensor([0.4400, 0.9500, 0.7100, 0.6200, 0.6400, 0.4700, 0.7300, 0.9600, 0.7100,
        0.7800, 0.5200, 0.7300, 0.8000, 0.8100, 0.5300, 0.6200, 0.7000, 0.7900,
        0.8700, 0.8900, 0.9200, 0.5700, 0.7300, 0.4400, 0.7400, 0.8400, 0.8400,
        0.6400, 0.9000, 0.7400, 0.7100, 0.4700, 0.9000, 0.6300, 0.7500, 0.3800,
        0.7200, 0.8000, 0.8400, 0.8200, 0.4800, 0.6400, 0.6200, 0.7500, 0.7300,
        0.4200, 0.9400, 0.7800, 0.7000, 0.9500, 0.7100, 0.6400, 0.7200, 0.8900,
        0.7000, 0.6800, 0.6400, 0.7700, 0.5700, 0.3400, 0.6100, 0.6700, 0.6100,
        0.8000, 0.6900, 0.5600, 0.6200, 0.4500, 0.8900, 0.6600, 0.7700, 0.9200,
        0.6900, 0.8800, 0.8200, 0.7500, 0.9400, 0.5200, 0.9300, 0.6400, 0.6800,
        0.7100, 0.5600, 0.8100, 0.7000, 0.5300, 0.5700, 0.7300, 0.8700, 0.7000,
        0.4700, 0.8400, 0.8000, 0.9300, 0.8600, 0.8300, 0.6200, 0.7100, 0.9400,
        0.6100, 0.8500, 0.8100, 0.8200, 0.3900, 0.8000, 0.6200, 0.9200, 0.6100,
        0.5200, 0.9100, 0.5400, 0.8500, 

In [18]:
y_train_tensor.shape

torch.Size([320])

Pytorch Starts!!!

In [19]:
#Dataset and DataLoader
from torch.utils.data import Dataset,DataLoader

In [20]:
class CustomDataset(Dataset):

  def __init__(self,x_tensor,y_tensor):
    self.x=x_tensor
    self.y=y_tensor

  def __len__(self):
    return len(self.x)

  def __getitem__(self,index):
    return (self.x[index],self.y[index])

In [37]:
train_dataset = CustomDataset(X_train_tensor,y_train_tensor)
test_dataset = CustomDataset(X_test_tensor,y_test_tensor)

In [38]:
len(test_dataset)

80

In [39]:
train_dataset[0]

(tensor([301.0000,  97.0000,   2.0000,   3.0000,   3.0000,   7.8800,   1.0000]),
 tensor(0.4400))

In [40]:
train_dataloader = DataLoader(train_dataset,batch_size=16,shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=16,shuffle=True)

In [41]:
print(train_dataloader)

In [42]:
for X,Y in train_dataloader:
  print(X)
  print(Y)
  print("."*50)

tensor([[302.0000, 102.0000,   3.0000,   3.5000,   5.0000,   8.3300,   0.0000],
        [322.0000, 115.0000,   5.0000,   4.0000,   4.5000,   9.3600,   1.0000],
        [308.0000, 106.0000,   3.0000,   3.5000,   2.5000,   8.2100,   1.0000],
        [312.0000,  98.0000,   1.0000,   3.5000,   3.0000,   8.1800,   1.0000],
        [329.0000, 114.0000,   2.0000,   2.0000,   4.0000,   8.5600,   1.0000],
        [331.0000, 115.0000,   5.0000,   4.0000,   3.5000,   9.4400,   1.0000],
        [314.0000, 108.0000,   3.0000,   4.5000,   3.5000,   8.1400,   0.0000],
        [326.0000, 112.0000,   4.0000,   4.0000,   3.5000,   9.1200,   1.0000],
        [313.0000, 104.0000,   3.0000,   4.0000,   4.5000,   8.6500,   0.0000],
        [314.0000, 107.0000,   2.0000,   2.5000,   4.0000,   8.5600,   0.0000],
        [303.0000,  98.0000,   1.0000,   2.0000,   2.5000,   7.6500,   0.0000],
        [315.0000, 107.0000,   2.0000,   4.0000,   3.0000,   8.5000,   1.0000],
        [300.0000, 100.0000,   3.0000,  

In [22]:
#Defining our model using Pytorch
import torch.nn as nn

class Model(nn.Module):

  def __init__(self,input_features):
    super().__init__()
    self.layers=nn.Sequential(
        nn.Linear(input_features,out_features=5),
        nn.ReLU(),
        nn.Linear(in_features=5,out_features=1),

    )

  def forward(self,features):
      return self.layers(features)

In [23]:
#Important Parameters
learning_rate = 0.1
epochs = 25
loss_function = nn.MSELoss()

Training Pipeline

In [24]:
model = Model(X_train_tensor.shape[1])
y_pred=model(X_train_tensor)
y_pred

tensor([[31.4811],
        [33.8638],
        [30.6616],
        [31.2323],
        [32.8495],
        [31.8891],
        [33.2415],
        [33.7127],
        [33.1507],
        [32.7665],
        [30.7563],
        [32.9380],
        [33.0433],
        [33.6181],
        [30.9925],
        [31.8446],
        [33.6868],
        [31.8574],
        [34.0169],
        [32.9419],
        [32.7954],
        [32.9759],
        [31.9145],
        [31.2528],
        [32.3963],
        [34.0536],
        [32.1343],
        [31.6128],
        [33.6331],
        [33.5440],
        [34.6665],
        [29.8386],
        [33.5314],
        [32.2727],
        [32.8797],
        [31.0639],
        [32.3178],
        [33.3681],
        [32.7527],
        [33.3714],
        [31.1898],
        [32.8713],
        [31.0930],
        [32.2859],
        [33.6647],
        [32.3448],
        [34.1718],
        [33.7441],
        [31.3568],
        [33.8117],
        [32.9958],
        [32.6816],
        [32.

In [43]:
#initialize a model
model = Model(X_train_tensor.shape[1])

#define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(epochs):

  for batch_x,batch_y in train_dataloader:

    y_pred=model(batch_x)
    loss=loss_function(y_pred,batch_y.unsqueeze(1))

    #clear gradients
    optimizer.zero_grad()

    #backward pass
    loss.backward()

    #parameter update
    optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item():.4f}')




Epoch: 1, Loss: 4653555.0000
Epoch: 2, Loss: 617.5881
Epoch: 3, Loss: 0.1009
Epoch: 4, Loss: 0.0298
Epoch: 5, Loss: 0.0182
Epoch: 6, Loss: 0.0202
Epoch: 7, Loss: 0.0351
Epoch: 8, Loss: 0.0215
Epoch: 9, Loss: 0.0202
Epoch: 10, Loss: 0.0172
Epoch: 11, Loss: 0.0229
Epoch: 12, Loss: 0.0256
Epoch: 13, Loss: 0.0216
Epoch: 14, Loss: 0.0164
Epoch: 15, Loss: 0.0192
Epoch: 16, Loss: 0.0234
Epoch: 17, Loss: 0.0191
Epoch: 18, Loss: 0.0062
Epoch: 19, Loss: 0.0156
Epoch: 20, Loss: 0.0102
Epoch: 21, Loss: 0.0205
Epoch: 22, Loss: 0.0123
Epoch: 23, Loss: 0.0215
Epoch: 24, Loss: 0.0234
Epoch: 25, Loss: 0.0098


Model Testing & Evaluation

In [44]:
# model evaluation
with torch.no_grad():
    y_pred = model(X_test_tensor)
    mse = torch.mean((y_pred.squeeze() - y_test_tensor) ** 2)
    rmse = torch.sqrt(mse)
    mae = torch.mean(torch.abs(y_pred.squeeze() - y_test_tensor))

print(f"MSE: {mse.item():.4f}")
print(f"RMSE: {rmse.item():.4f}")
print(f"MAE: {mae.item():.4f}")

MSE: 0.0234
RMSE: 0.1530
MAE: 0.1229
